In [7]:
import pandas as pd
import re
import numpy as np
from liquid import liquids
from garnish import garnishes
from boston_functions import *

data = pd.read_csv("./mr-boston-all-glasses.csv")

data = data[data.loc[:, "glass-size"].notna()]

valid_units = ["oz", "tsp", "splash", "dash"]

fill_liquid = ["ginger ale", "carbonated water", "cola", "water", "chilled champagne", "soda water", 
               "club soda", "ginger ale or soda water", "lemon-lime soda", "ginger beer", "bitter lemon soda",
               "apple juice", "orange juice"]

invaild_ingredients = ['chopped', 'cut in half', 'cut in halves', 'cut into halves', 'flamed', 'hulled', 'long', 'skinned',
                       'whipped', "preferably b.a. reynold's", 'preferably jamaican', 'preferably japanese', 
                       'preferably pedro ximenez', "such as bittermen's elemakule", 'such as demerara', 
                       'such as islay or skye', 'such as nasturtium']

ingredient_indicies = range(3, 14)

In [24]:
data.head()

,name,category,measurement-1,ingredient-1,measurement-2,ingredient-2,measurement-3,ingredient-3,measurement-4,ingredient-4,measurement-5,ingredient-5,measurement-6,ingredient-6,instructions,glass,glass-size
0,Gauguin,Cocktail Classics,2 oz,Light Rum,1 oz,Passion Fruit Syrup,1 oz,Lemon Juice,1 oz,Lime Juice,NaN,NaN,NaN,NaN,Combine ingredients with a cup of crushed ice ...,Old-Fashioned Glass,6 to 8 ounces
1,Fort Lauderdale,Cocktail Classics,1 1/2 oz,Light Rum,1/2 oz,Sweet Vermouth,1/4 oz,Juice of Orange,1/4 oz,Juice of a Lime,NaN,NaN,NaN,NaN,Shake with ice and strain into old-fashioned g...,Old-Fashioned Glass,6 to 8 ounces
2,Apple Pie,Cordials and Liqueurs,3 oz,Apple schnapps,1 oz,Cinnamon schnapps,NaN,Apple slice,NaN,NaN,NaN,NaN,NaN,NaN,Pour into ice-filled old-fashioned glass. Garn...,Old-Fashioned Glass,6 to 8 ounces
3,Cuban Cocktail No. 1,Cocktail Classics,1/2 oz,Juice of a Lime,1/2 oz,Powdered Sugar,2 oz,Light Rum,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces
4,Cool Carlos,Cocktail Classics,1 1/2 oz,Dark rum,2 oz,Cranberry Juice,2 oz,Pineapple Juice,1 oz,Orange curacao,1 oz,Sour Mix,NaN,NaN,"Mix all ingredients except curacao with ice, s...",Collins Glass,14 to 16 ounces


In [29]:
glass_size = data.loc[:, "glass-size"]

def get_size(string):
    string = remove_leading_space(string)
    num = ""
    for char in string:
        if char in [str(n) for n in range(10)]:
            num += char
        elif len(num) > 0:
            return eval(num)
    return string
    
        
get_size("test")

'test'

In [19]:
# liquid_generational = get_word_children(liquids)
def get_word_children(list_of_words):
    if len(list_of_words) == 0:
        return []
    else:
        children = []
        for child in list_of_words:
            for parent in list_of_words:
                if child != parent and child in parent:
                    children.append(child)
        children = list(set(children))
        parents = []
        for word in list_of_words:
            if word not in children:
                parents.append(word)
        parents = list(set(parents))
        return [parents] + get_word_children(children)

# get_word_children(liquids)

def find_ingredient(ingredient, ingredient_lists):
    returned_ingredient = None
    for ingredient_list in ingredient_lists:
        for possible_match in ingredient_list:
            if ingredient in possible_match:
                returned_ingredient = possible_match
                break
        if returned_ingredient is not None:
            break
    return returned_ingredient

In [30]:
def get_cocktail_recipies(dataframe, ingredient_indicies, liquid_list, garnish_list, invaild_ingredients, valid_units):

    all_recipies = []
    measures = []
    
    liquid_generational = get_word_children(liquid_list)
    garnish_generational = get_word_children(garnish_list)
    
    # For each row in the data
    for row in range(len(dataframe)):
        
        name = dataframe.iloc[row, 0]
        category = dataframe.iloc[row, 1]
        instructions = dataframe.iloc[row, 14]
        glass = dataframe.iloc[row, 15]
        glass_size = get_size(dataframe.iloc[row, 16])
        drink_recipe = []
        
        # Pull out the ingredients and their measurements
        for ingredient_index in ingredient_indicies:

            # Ingredients
            ingredients = mung_text(str(dataframe.iloc[row, ingredient_index]))
            ingredient_list = split_on_comma(ingredients)

            # Potential Measure for that ingredient
            measure = str(dataframe.iloc[row, ingredient_index - 1])
            valid_measure = False
            if len(measure) < 10:
                for unit in valid_units:
                    if unit in measure:
                        valid_measure = True
                        
#             if not valid_measure and measure[0] in [str(n) for n in range(10)]:
#                 valid_measure = True
#                 print(measure)
#                 measure = str(measure[0]) + " oz*"
#                 print(measure)
                
            if valid_measure:
                measures.append(measure)
            else:
                measure = "fill"

            # Find ingredient type must start at most complecated and move to least
            # Some garnishes like raspberry appear in liquids => liquids first
            for ingredient in ingredient_list:
                
                # Ignore invaild data
                if ingredient in invaild_ingredients:
                    pass
                
                else:
                    
                    # Try Liquid
                    recorded_ingredient = find_ingredient(ingredient, liquid_generational)
                    ingredient_type = "liquid"
                    
                    if recorded_ingredient is None:
                        
                        # Try Garnish
                        recorded_ingredient = find_ingredient(ingredient, garnish_generational)
                        ingredient_type = "garnish"
                        
                    if recorded_ingredient is None:
                        # Missed a bad ingredient pass
                        pass
                    
                    else:
                        if ingredient_type == "garnish":
                            # no need for measure
                            drink_recipe += [["add", recorded_ingredient]]
                        else:
                            drink_recipe += [[measure, recorded_ingredient]]
                            
        drink_info = {"name": name,
                      "category": category,
                      "instructions": instructions,
                      "glass": glass,
                      "glass_size": glass_size,
                      "recipe": drink_recipe}
        all_recipies.append(drink_info)
        
    return all_recipies, measures

                        
all_recipies, measures = get_cocktail_recipies(data, ingredient_indicies, liquids, garnishes, invaild_ingredients, valid_units)

In [31]:
all_recipies

[{'name': 'Gauguin',
  'category': 'Cocktail Classics',
  'instructions': 'Combine ingredients with a cup of crushed ice in blender and blend at low speed. Serve in old-fashioned glass. Top with a cherry.',
  'glass': 'Old-Fashioned Glass',
  'glass_size': 6,
  'recipe': [['2 oz', 'batavia arrack or light rum'],
   ['1 oz', 'passion fruit syrup'],
   ['1 oz', 'fresh lemon juice'],
   ['1 oz', "rose's lime juice"]]},
 {'name': 'Fort Lauderdale',
  'category': 'Cocktail Classics',
  'instructions': 'Shake with ice and strain into old-fashioned glass over ice cubes. Add a slice of orange.',
  'glass': 'Old-Fashioned Glass',
  'glass_size': 6,
  'recipe': [['1 1/2 oz', 'batavia arrack or light rum'],
   ['1/2 oz', 'sweet vermouth'],
   ['1/4 oz', 'each orange juice and sweet sherry'],
   ['1/4 oz', "rose's lime juice"]]},
 {'name': 'Apple Pie',
  'category': 'Cordials and Liqueurs',
  'instructions': 'Pour into ice-filled old-fashioned glass. Garnish with apple and top with cinnamon.',
  '

In [ ]:
# def remove_duplicates(word_list):
#     result = list(set(word_list))
#     length = len(word_list)
#     for i1 in range(length-1):
#         for i2 in range(i1 + 1, length):
#             word1 = word_list[i1]
#             word2 = word_list[i2]
#             if word1 == word2 and word1 in result:
#                 result.remove(word1)
#     return result

# def get_unique_words(string_list):
#     words = []
#     for string in string_list:
#         words += string.split(" ")
#     return remove_duplicates(words)
    
# get_unique_words(['light rum', 'passion fruit syrup', 'lemon juice', 'lime juice'])
# # for word_list in r:
# #     print(get_un)

In [ ]:
def adjust_measurement(measurement):
    items = measurement.split(' ')
    final = ''
    amount = 0
    for item in items:
        try:
            amount += eval(item)
        except:
            final += f"{item} "
    if amount > 0:
        final = f"{round(amount, 2)} " + final
    return final[:-1]